# Title of Notebook
> Subtitle goes here

In [4]:
from nbdev import *

In [17]:
import requests
import pandas as pd
import os
import json
from datetime import datetime
import pytz  # For working with time zones


headers = {
    'X-PW-AccessToken':os.environ["COPPER_API_KEY"],
    'X-PW-Application':'developer_api',
    'X-PW-UserEmail':'cooper@rehabpath.com',
    'Content-Type':'application/json'
}

def refresh_custom_fields(return_results:bool = False, #Whether you would like results returned
):
    """
    Function to refresh the custom_field_json file and related dictionary. Option to 
    return resulting dictionary.
    """
    # Fetching custom fields:
    list_fields = requests.get('https://api.copper.com/developer_api/v1/custom_field_definitions',headers=headers)
    
    # Was the API request a success?
    if list_fields.status_code != 200:
        print('Error fetching custom fields.')
        if return_results:
            print('Returning previous version.')
            with open('custom_fields.json', encoding="utf-8") as f:
                return json.load(f)
    
    # Obtaining response text and initializing dictionary:
    list_fields_json = list_fields.json()
    custom_field_dict = {}

    for item in list_fields_json:
        item_dict = {}
        for sub_item in ['name','data_type','available_on','is_filterable']:
            item_dict[sub_item] = item[sub_item] 

        item_options = {}

        if 'options' in item:
            for sub_item in item['options']:
                sub_item_name = sub_item['name']
                sub_item_id = sub_item['id']

                item_options[sub_item_id] = sub_item_name
        
            item_dict['options'] = item_options
        item_id = item['id']
        custom_field_dict[item_id] = item_dict
    
    # Updating local file:
    with open('custom_fields.json', 'w') as file:
        json.dump(custom_field_dict, file, indent=4)
    
    #Option to return results or just update local file:
    if return_results == True:
        return custom_field_dict
    else:
        print('Successfully Updated File!')
        pass

custom_fields = refresh_custom_fields(True)

def get_cf_info(cf_id:str,     #ID of custom field
                cf_info:list,  #Designed information about field, list if multiple items
)->list: #Returns list if cf_info is list. Otherwise, returns value
    """
    Function to get the custom field information based on the field id. Can be supplied with a sigular value for 'cf_info' or multiple as a list.
    """

    if isinstance(cf_id,str):
        cf_id = int(cf_id)      

    # Check if cf_info item(s) are valid
    if isinstance(cf_info, list):
        for item_name in cf_info:
            # Remove faulty items:
            if item_name not in ['name','data_type','available_on','is_filterable','options']:
                cf_info.pop(item_name)
                print(f'Invalid cf_info: {cf_info}')
            
            # Cancel function if no valid items
            if len(cf_info) == 0:
                pass
    elif cf_info not in ['name','data_type','available_on','is_filterable','options']:
        print(f'Invalid cf_info: {cf_info}'); pass

    if isinstance(cf_info,list):
        return_list = []
        for item_name in cf_info:
            value = custom_fields[cf_id].get(item_name)
            return_list.append(value)
        return return_list
    else:
        value = custom_fields[cf_id].get(cf_info)
        return value

def get_cf_options(cf_id:str, #Coppper ID for custom field
)->dict: # Returns dictionary of options for field
    return custom_fields.get(cf_id).get('options')

def cf_option_name(cf_id:str, #Coppper ID for custom field
option_id:str, #Coppper ID for option
)->str: #Returns name/value of option
    return get_cf_options(cf_id).get(option_id)

def reformat_company_data(company_data:list, # Company data as a list of dictonaries
):
    """
    Function to reformat the company data recieved from Copper in their JSON response
    """

    native_items = ['id', 'name', 'address', 'assignee_id', 'contact_type_id']

    output_dict = {}

    for item in native_items:
        output_dict[item] = company_data.get(item, None)
    
    custom_field_data = company_data['custom_fields']

    for dict_item in custom_field_data:
        item_id = dict_item['custom_field_definition_id']
        item_name = custom_fields_dict.get(item_id, None)

        if item_name not in Custom_Fields or item_name is None:
            continue
        
        if item_name is not None:
            item_value = dict_item['value']
            output_dict[item_name] = item_value
    
    return output_dict


In [10]:
show_doc(refresh_custom_fields)

---

### refresh_custom_fields

>      refresh_custom_fields (return_results:bool=False)

Function to refresh the custom_field_json file and related dictionary. Option to 
return resulting dictionary.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| return_results | bool | False | Whether you would like results returned |

In [16]:
show_doc(get_cf_info)

---

### get_cf_info

>      get_cf_info (cf_id:str, cf_info:list)

Function to get the custom field information based on the field id. Can be supplied with a sigular value for 'cf_info' or multiple as a list

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| cf_id | str | ID of custom field |
| cf_info | list | Designed information about field, list if multiple items |
| **Returns** | **list** | **Returns list if cf_info is list. Otherwise, returns value** |

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=00c552c7-bcb7-4815-8ee3-8291f5703f52' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>